BackgroundSubtractorMOG2 usa um método para modelar cada pixel de fundo por uma mistura de distribuições K Gaussianas
Os pesos da mistura representam as proporções de tempo que essas cores permanecem na cena. 
As cores de fundo prováveis são as que ficam mais longas e mais estáticas.

In [3]:
import numpy as np
import cv2

cap = cv2.VideoCapture('sample_41.mp4')

fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    sucess, frame = cap.read()
    
    if not sucess:
        break

    fgmask = fgbg.apply(frame)

    
    cv2.imshow('frame',fgmask)
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break

cap.release()

Fluxo óptico é o padrão de movimento aparente de objetos de imagem entre dois quadros consecutivos causados pelo movimento do objeto ou da câmera. É um campo vetorial 2D onde cada vetor é um vetor de deslocamento mostrando o movimento dos pontos do primeiro quadro ao segundo. 


In [5]:
import numpy as np
import cv2

cap = cv2.VideoCapture('sample_41.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 20,
                       qualityLevel = 0.01,
                       minDistance = 15,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    sucess,frame = cap.read()
    
    if not sucess:
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()
